# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weathercity_df = "output/weather_cities.csv"
part1_df = pd.read_csv(weathercity_df)
part1_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,sinnar,100,SD,1619813099,16,13.5691,33.5672,95.04,5.01
1,punta arenas,90,CL,1619813099,93,-53.1500,-70.9167,42.80,6.91
2,gravdal,40,NO,1619813099,80,68.1214,13.5031,32.00,14.97
3,ushuaia,75,AR,1619812808,81,-54.8000,-68.3000,41.00,1.01
4,nikolskoye,0,RU,1619813100,45,59.7035,30.7861,41.00,4.47
...,...,...,...,...,...,...,...,...,...
571,ngunguru,100,NZ,1619813280,95,-35.6167,174.5000,54.00,1.99
572,hirara,100,JP,1619813232,91,24.8000,125.2833,74.52,15.57
573,praya,40,CV,1619813281,73,14.9215,-23.5087,71.60,17.27
574,belaya gora,100,RU,1619813281,91,68.5333,146.4167,22.24,10.02


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Create a map using coordinates 
marker_locations = part1_df[['Lat', 'Lng']]
weights = part1_df['Humidity'].astype(float)
# Plot Heatmap
fig = gmaps.figure(zoom_level = 1.8, center = [10,50])

# Create heat layer
fig.add_layer(gmaps.heatmap_layer(marker_locations,  weights= weights,
                                 dissipating=False, max_intensity=100,
                                 point_radius=2.2))

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
#Fitting my weather criteria:
high_temp = 80
low_temp = 70
new_df = part1_df.loc[(part1_df['Max Temp']< high_temp) & (part1_df['Max Temp'] > low_temp)        
                                                        & (part1_df['Cloudiness'] <= 6)
                                                        & (part1_df['Wind Speed'] < 24)
                                                        & (part1_df['Humidity'] > 35)
                                                        & (part1_df['Humidity'] < 70),:]        
new_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
64,mar del plata,0,AR,1619813121,60,-38.0023,-57.5575,71.01,12.66
81,pisco,0,PE,1619813126,64,-13.7000,-76.2167,73.40,13.80
105,bell ville,0,AR,1619813134,36,-32.6259,-62.6887,75.99,15.01
115,noumea,0,NC,1619813139,68,-22.2763,166.4572,75.00,2.30
204,gat,0,IL,1619813105,43,31.6100,34.7642,73.99,6.31
270,marsh harbour,6,BS,1619813128,65,26.5412,-77.0636,79.63,9.64
275,sao joao da barra,0,BR,1619813191,64,-21.6403,-41.0511,75.20,8.05
302,rio grande,0,BR,1619813198,64,-32.0350,-52.0986,73.40,11.50
303,punta alta,1,AR,1619812965,40,-38.8759,-62.0736,72.18,16.89
433,sinjar,1,IQ,1619813237,40,36.3209,41.8754,72.81,7.25


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
hotel_df = pd.DataFrame(new_df)
hotel_df['Hotel Name'] = ''
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
64,mar del plata,0,AR,1619813121,60,-38.0023,-57.5575,71.01,12.66,
81,pisco,0,PE,1619813126,64,-13.7000,-76.2167,73.40,13.80,
105,bell ville,0,AR,1619813134,36,-32.6259,-62.6887,75.99,15.01,
115,noumea,0,NC,1619813139,68,-22.2763,166.4572,75.00,2.30,
204,gat,0,IL,1619813105,43,31.6100,34.7642,73.99,6.31,
270,marsh harbour,6,BS,1619813128,65,26.5412,-77.0636,79.63,9.64,
275,sao joao da barra,0,BR,1619813191,64,-21.6403,-41.0511,75.20,8.05,
302,rio grande,0,BR,1619813198,64,-32.0350,-52.0986,73.40,11.50,
303,punta alta,1,AR,1619812965,40,-38.8759,-62.0736,72.18,16.89,
433,sinjar,1,IQ,1619813237,40,36.3209,41.8754,72.81,7.25,


In [21]:
from pprint import pprint 
# find the closest hotel of each type to coordinates
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#setting parameters
params = {
        "type": "hotel",
        "radius": 5000,
        "key": g_key
         }
# Use the lat/lng recovered to identify hotels
for index, row in hotel_df.iterrows():
    # Get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Change locations
    params["location"] = f"{lat},{lng}"
    
    # Make API request and convert to json
    response_hotel = requests.get(base_url, params=params).json()
    
    # Print the json (pretty printed) 
    # pprint(response_hotel)
    results = response_hotel["results"]
    # Try to grab the hotel name if is available in the API
    try:
        print(f"The first Hotel result for location : {lat},{lng} is: Hotel {results[0]['name']}.")
        #Storing the first Hotel result into the DataFrame
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name'] 
    # Handle exceptions
    except KeyError as e:
        message ='Key Error: ' + str(e)
        print(message)
        print("Skipping...")
    except IndexError as e:
        message ='Key Error: ' + str(e)
        print(message)
        print("Skipping...")
print("------------")

The first Hotel result for location : -38.0023,-57.5575 is: Hotel Mar del Plata.
The first Hotel result for location : -13.7,-76.2167 is: Hotel Pisco.
The first Hotel result for location : -32.6259,-62.6887 is: Hotel Bell Ville.
The first Hotel result for location : -22.2763,166.4572 is: Hotel Nouméa.
The first Hotel result for location : 31.61,34.7642 is: Hotel Kiryat Gat.
The first Hotel result for location : 26.5412,-77.0636 is: Hotel Marsh Harbour.
The first Hotel result for location : -21.6403,-41.0511 is: Hotel Grussaí.
The first Hotel result for location : -32.035,-52.0986 is: Hotel Rio Grande.
The first Hotel result for location : -38.8759,-62.0736 is: Hotel Punta Alta.
The first Hotel result for location : 36.3209,41.8754 is: Hotel Sinjar.
------------


In [22]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
64,mar del plata,0,AR,1619813121,60,-38.0023,-57.5575,71.01,12.66,Mar del Plata
81,pisco,0,PE,1619813126,64,-13.7000,-76.2167,73.40,13.80,Pisco
105,bell ville,0,AR,1619813134,36,-32.6259,-62.6887,75.99,15.01,Bell Ville
115,noumea,0,NC,1619813139,68,-22.2763,166.4572,75.00,2.30,Nouméa
204,gat,0,IL,1619813105,43,31.6100,34.7642,73.99,6.31,Kiryat Gat
270,marsh harbour,6,BS,1619813128,65,26.5412,-77.0636,79.63,9.64,Marsh Harbour
275,sao joao da barra,0,BR,1619813191,64,-21.6403,-41.0511,75.20,8.05,Grussaí
302,rio grande,0,BR,1619813198,64,-32.0350,-52.0986,73.40,11.50,Rio Grande
303,punta alta,1,AR,1619812965,40,-38.8759,-62.0736,72.18,16.89,Punta Alta
433,sinjar,1,IQ,1619813237,40,36.3209,41.8754,72.81,7.25,Sinjar


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content= hotel_info)
# Add the layer to the map
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

#### Screenshot of Heat Map 
![map.png](Images/map.png)